In [ ]:
# import time
# import cv2
# import numpy as np
# import pandas as pd
# from ML_KEM import ML_KEM
# from AES_CTR.AES_Lib import AES_Software

# # --- CẤU HÌNH TEST ĐA ĐỘ PHÂN GIẢI ---
# # List các độ phân giải cần test (Width, Height)
# TEST_RESOLUTIONS = [
#     (320, 240),   # QVGA (Nhẹ)
#     (640, 480),   # VGA (Tiêu chuẩn)
#     (1280, 720)   # HD 720p (Nặng - Stress Test)
# ]

# TEST_FRAMES = 50       # Số frame test cho mỗi độ phân giải
# WARMUP_FRAMES = 5      # Số frame chạy đà
# CAMERA_INDEX = 0

# print(f"=== COMPREHENSIVE SYSTEM BENCHMARK ===")
# print(f"Test Resolutions: {TEST_RESOLUTIONS}")

# # 1. SETUP CAMERA & KYBER (Chỉ làm 1 lần)
# cap = cv2.VideoCapture(CAMERA_INDEX)
# if not cap.isOpened():
#     raise RuntimeError("Error: Cannot access Camera!")

# # Kyber Key Exchange (Làm 1 lần dùng chung cho cả bài test)
# print("\n[INIT] Performing Kyber Key Exchange...")
# pk, sk = ML_KEM.KeyGen()
# ss_enc, c = ML_KEM.Encaps(pk)
# aes_engine = AES_Software(ss_enc)
# print("-> Secure Channel Established.")

# # Bảng tổng hợp kết quả
# final_report = []

# # ==========================================
# # 2. MULTI-RESOLUTION LOOP
# # ==========================================
# for res in TEST_RESOLUTIONS:
#     target_w, target_h = res
#     print(f"\n>> TESTING RESOLUTION: {target_w}x{target_h}...")
    
#     # Tính toán kích thước payload lý thuyết (cho báo cáo)
#     # Ảnh màu 3 kênh (RGB) => W * H * 3 bytes
#     payload_bytes = target_w * target_h * 3
#     payload_mb = payload_bytes / (1024 * 1024)
    
#     # Reset biến đo lường
#     latencies = {"Encrypt": [], "Decrypt": [], "Total": []}
    
#     # --- WARM UP ---
#     for _ in range(WARMUP_FRAMES):
#         ret, frame = cap.read()
#         if ret:
#             frame_resized = cv2.resize(frame, res)
#             enc = aes_engine.encrypt_image(frame_resized)
#             _ = aes_engine.decrypt_to_image(enc, (target_h, target_w, 3), frame.dtype)

#     # --- MEASUREMENT ---
#     start_time_res = time.perf_counter()
    
#     for i in range(TEST_FRAMES):
#         # 1. Capture & Resize (Để đảm bảo đúng kích thước input mong muốn)
#         # Lưu ý: Ta dùng resize phần mềm để ép input đúng chuẩn test, 
#         # thay vì set camera (vì nhiều cam không hỗ trợ đúng số lẻ)
#         ret, frame = cap.read()
#         if not ret: break
        
#         frame_input = cv2.resize(frame, res)
        
#         # 2. Encrypt Measurement
#         t1 = time.perf_counter()
#         encrypted_data = aes_engine.encrypt_image(frame_input)
#         t_enc = time.perf_counter() - t1
        
#         # 3. Decrypt Measurement
#         t2 = time.perf_counter()
#         decrypted_frame = aes_engine.decrypt_to_image(encrypted_data, (target_h, target_w, 3), frame_input.dtype)
#         t_dec = time.perf_counter() - t2
        
#         latencies["Encrypt"].append(t_enc)
#         latencies["Decrypt"].append(t_dec)
#         latencies["Total"].append(t_enc + t_dec)

#     total_duration = time.perf_counter() - start_time_res
#     real_fps = TEST_FRAMES / total_duration
    
#     # Tính trung bình (ms)
#     avg_enc = np.mean(latencies["Encrypt"]) * 1000
#     avg_dec = np.mean(latencies["Decrypt"]) * 1000
#     avg_total = np.mean(latencies["Total"]) * 1000
    
#     # Tính Throughput (MB/s) dựa trên tốc độ xử lý AES (Encrypt)
#     # Throughput = Payload / Avg_Encrypt_Time_in_Sec
#     throughput_mb_s = payload_mb / (avg_enc / 1000)

#     # Lưu vào báo cáo
#     final_report.append({
#         "Resolution": f"{target_w}x{target_h}",
#         "Payload (MB)": round(payload_mb, 2),
#         "Avg Encrypt (ms)": round(avg_enc, 2),
#         "Avg Decrypt (ms)": round(avg_dec, 2),
#         "Est. Throughput (MB/s)": round(throughput_mb_s, 2),
#         "Real System FPS": round(real_fps, 2)
#     })

# cap.release()

# # ==========================================
# # 3. FINAL COMPARISON REPORT
# # ==========================================
# df_report = pd.DataFrame(final_report)
# print("\n" + "="*50)
# print("      MULTI-RESOLUTION PERFORMANCE REPORT      ")
# print("="*50)
# print(df_report.to_string(index=False))

# # Vẽ biểu đồ so sánh trực quan
# try:
#     import matplotlib.pyplot as plt
    
#     # Biểu đồ Latency
#     df_report.plot(x="Resolution", y=["Avg Encrypt (ms)", "Avg Decrypt (ms)"], kind="bar", figsize=(8, 5))
#     plt.title("AES Processing Latency by Resolution")
#     plt.ylabel("Time (ms)")
#     plt.xticks(rotation=0)
#     plt.grid(axis='y', linestyle='--')
#     plt.show()
    
#     # Biểu đồ Throughput
#     df_report.plot(x="Resolution", y="Est. Throughput (MB/s)", kind="line", marker='o', color='red', figsize=(8, 5))
#     plt.title("System Throughput (MB/s) vs Payload Size")
#     plt.ylabel("Throughput (MB/s)")
#     plt.grid(True)
#     plt.show()
# except ImportError:
#     pass

In [ ]:
import time
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ML_KEM import ML_KEM
from AES_CTR.AES_Lib import AES_Software

# --- CẤU HÌNH TEST CHÍNH THỨC ---
KYBER_ITERATIONS = 1000     # Số lần lặp đo Kyber (Chuẩn: 1000 hoặc 10000)
AES_TEST_FRAMES = 100       # Số frame test cho mỗi độ phân giải
AES_RESOLUTIONS = [
    (320, 240),    # QVGA
    (640, 480),    # VGA 
    (1280, 720)    # HD
]

print("=== PHASE 1: COMPREHENSIVE PERFORMANCE BENCHMARK ===")
print(f"Configurations: Kyber N={KYBER_ITERATIONS}, AES Frames={AES_TEST_FRAMES}")

# ========================================================
# SECTION 1: KYBER MICRO-BENCHMARK (Official Metrics)
# Đo 3 metric quan trọng: KeyGen, Encaps, Decaps
# ========================================================
print(f"\n>> [1/2] MEASURING KYBER METRICS...")

# Mảng lưu thời gian (đơn vị: ms)
kyber_times = {"KeyGen": [], "Encaps": [], "Decaps": []}

# Warm-up (Chạy nháp để nạp cache)
print("   -> Warming up...", end="")
pk, sk = ML_KEM.KeyGen()
ss, c = ML_KEM.Encaps(pk)
_ = ML_KEM.Decaps(sk, c)
print(" Done.")

print("   -> Running iterations...", end="")
for _ in range(KYBER_ITERATIONS):
    # 1. KeyGen
    t0 = time.perf_counter()
    pk, sk = ML_KEM.KeyGen()
    kyber_times["KeyGen"].append((time.perf_counter() - t0) * 1000)

    # 2. Encaps
    t0 = time.perf_counter()
    ss_sender, c = ML_KEM.Encaps(pk)
    kyber_times["Encaps"].append((time.perf_counter() - t0) * 1000)

    # 3. Decaps
    t0 = time.perf_counter()
    ss_receiver = ML_KEM.Decaps(sk, c)
    kyber_times["Decaps"].append((time.perf_counter() - t0) * 1000)
print(" Done.")

# Tổng hợp số liệu Kyber
df_kyber = pd.DataFrame(kyber_times)
print("\n[TABLE 1] KYBER PERFORMANCE REPORT (Unit: ms)")
print(df_kyber.describe().T[['mean', 'std', 'min', 'max']].round(4))

# ========================================================
# SECTION 2: AES MULTI-RESOLUTION BENCHMARK
# Đo Encrypt/Decrypt Latency & Throughput
# ========================================================
print(f"\n>> [2/2] MEASURING AES SCALABILITY...")

aes_results = []
aes_engine = AES_Software(ss_sender) # Dùng khóa sinh ra từ Kyber

# Setup Camera (Nếu có)
cap = cv2.VideoCapture(0)
has_camera = cap.isOpened()
if not has_camera: print("   ! Warning: No Camera found. Using Synthetic Data.")

for res in AES_RESOLUTIONS:
    w, h = res
    print(f"   -> Testing Resolution {w}x{h}...", end="")
    
    # Chuẩn bị input (Dùng ảnh thật hoặc giả lập)
    if not has_camera:
        input_frame = np.random.randint(0, 256, (h, w, 3), dtype=np.uint8)
    
    # Warm-up AES cho resolution này
    dummy = np.zeros((h, w, 3), dtype=np.uint8)
    aes_engine.decrypt_to_image(aes_engine.encrypt_image(dummy), (h,w,3), np.uint8)

    t_enc_list = []
    t_dec_list = []

    for _ in range(AES_TEST_FRAMES):
        if has_camera:
            ret, raw = cap.read()
            if ret: input_frame = cv2.resize(raw, res)
            else: break
        
        # Đo Encrypt
        t0 = time.perf_counter()
        enc_data = aes_engine.encrypt_image(input_frame)
        t_enc_list.append((time.perf_counter() - t0) * 1000)

        # Đo Decrypt
        t0 = time.perf_counter()
        dec_img = aes_engine.decrypt_to_image(enc_data, input_frame.shape, input_frame.dtype)
        t_dec_list.append((time.perf_counter() - t0) * 1000)

    # Tính toán Metrics
    avg_enc = np.mean(t_enc_list)
    avg_dec = np.mean(t_dec_list)
    payload_mb = (w * h * 3) / (1024 * 1024)
    throughput = payload_mb / (avg_enc / 1000) # MB/s
    
    aes_results.append({
        "Resolution": f"{w}x{h}",
        "Payload (MB)": round(payload_mb, 2),
        "Encrypt (ms)": round(avg_enc, 3),
        "Decrypt (ms)": round(avg_dec, 3),
        "Throughput (MB/s)": round(throughput, 2)
    })
    print(" Done.")

if has_camera: cap.release()

df_aes = pd.DataFrame(aes_results)
print("\n[TABLE 2] AES PERFORMANCE ACROSS RESOLUTIONS")
print(df_aes.to_string(index=False))

# (Tùy chọn) Lưu ra CSV để vẽ biểu đồ trên Excel
# df_kyber.to_csv("phase1_kyber_metrics.csv")
# df_aes.to_csv("phase1_aes_metrics.csv")
print("\n=== BENCHMARK COMPLETED ===")